In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
def load_data():
    df = pd.read_csv('weather_data.csv')
    df['time1'] = df['date'] + ' ' + df['time']
    df = df.drop(columns={'Unnamed: 0', 'date', 'time'})
    df['time1'] =  pd.to_datetime(df['time1'], format='%m/%d/%Y %H:%M')
    df.rename(columns={ 'time1': "time" }, inplace = True)
    df = df.set_index('time')

    # drop dublicates
    df = df.loc[~df.index.duplicated(keep='first')]
    return df

In [ ]:
def replace_nan(df):
    # replace '-----' to nan
    df['T'].replace('-----', np.nan, inplace=True)

    # df['Tmax'].replace('-----', np.nan, inplace=True)
    # df['Tmin'].replace('-----', np.nan, inplace=True)
    # df['Td'].replace('-----', np.nan, inplace=True)

    df['T'] = df['T'].astype(float)
    # df['Tmax'] = df['Tmax'].astype(float)
    # df['Tmin'] = df['Tmin'].astype(float)
    # df['Td'] = df['Td'].astype(float)
    
    df['T'] = df['T'].where(df['T'].values == 999, other=(df['T'].fillna(method='ffill') + df['T'].fillna(method='bfill'))/2)
    df[df['T'].isnull()]
    
    return df

In [ ]:
def shift_rows(df):
    # dfao = dfao.append(pd.Series(name=datetime.datetime(2020, 5, 30, 9, 0)))
    # dfao = dfao.append(pd.Series(name=datetime.datetime(2020, 5, 30, 12, 0)))
    
    import datetime
    datetime_3 = df.iloc[-1:].index + datetime.timedelta(hours=3)
    datetime_6 = df.iloc[-1:].index + datetime.timedelta(hours=6)
    
    df = df.append(pd.DataFrame(index=datetime_3))
    df = df.append(pd.DataFrame(index=datetime_6))

    # shift for utc to omsk time change !!!
    df = df.shift(periods=2)
    df = df.iloc[2:]
    df.tail(5)
    return df

In [ ]:
# Serbakul
folder_city = 'Serbakul' # Sargatskoe # Serbakul # Kalacinsk
os.chdir("B:/Projects/CourseWork/coursework/triangle_ogimet/data/" + folder_city)
print(os.getcwd())

In [ ]:
df = load_data()
df = replace_nan(df)
df = shift_rows(df)
df_ser = df.copy()

In [ ]:
# Sargatskoe
folder_city = 'Sargatskoe' # Sargatskoe # Serbakul # Kalacinsk
os.chdir("B:/Projects/CourseWork/coursework/triangle_ogimet/data/" + folder_city)
print(os.getcwd())

In [ ]:
df = load_data()
df = replace_nan(df)
df = shift_rows(df)
df_sar = df.copy()

In [ ]:
# Kalacinsk
folder_city = 'Kalacinsk' # Sargatskoe # Serbakul # Kalacinsk
os.chdir("B:/Projects/CourseWork/coursework/triangle_ogimet/data/" + folder_city)
print(os.getcwd())

In [ ]:
df = load_data()
df = replace_nan(df)
df = shift_rows(df)
df_kal = df.copy()

In [ ]:
# Omsk
os.chdir("B:/Projects/CourseWork/coursework/triangle_ogimet/data/")
print(os.getcwd())

In [ ]:
df = load_data()
df = replace_nan(df)
df = shift_rows(df)
df_omsk = df.copy()

In [ ]:
df_sar = pd.DataFrame(df_sar['T'])
df_ser = pd.DataFrame(df_ser['T'])
df_omsk = pd.DataFrame(df_omsk[['T', 'ddd']])
df_omsk.rename(columns={"T": "omsk", "ddd":"ddd_omsk"}, inplace=True)

df_sar.rename(columns={"T": "sar"}, inplace=True)
df_ser.rename(columns={"T": "ser"}, inplace=True)

df_three = pd.concat([df_kal,df_sar, df_ser, df_omsk], axis=1)
df_three.rename(columns={"T": "kal"}, inplace=True)
df_three.drop(columns={'Td', 'Tmax', 'Tmin', 'ddd', 'ff', 'prec', 'nt', 'nh', 'h'}, inplace=True)
df_three['mean'] = df_three.mean(axis=1)

df_three['diff'] = df_three['omsk'] - df_three['mean']
df_three['abs_diff'] = abs(df_three['omsk'] - df_three['mean'])

In [ ]:
df_three.loc['2000-11-01':'2001-03-01'].tail(40) # # 28791 - Шербакуль # 28598 - Саргатское # 28696 - Калачинск # 28698 - Омск

In [ ]:
df_three_ = df_three.dropna()
df_three_.loc['2000-11-01':'2001-03-01'].tail(40)

In [ ]:
df_three_.groupby('ddd_omsk')['mean'].count()

In [ ]:
# rename wind columns value to 4 quarter
df_three_['ddd_omsk'].replace(['W','SW','WSW','WNW'],'West', inplace=True)
df_three_['ddd_omsk'].replace(['N','NW','NNW','NNE'],'North', inplace=True)
df_three_['ddd_omsk'].replace(['E','NE','ESE','ENE'],'East', inplace=True)
df_three_['ddd_omsk'].replace(['S','SE','SSW','SSE'],'South', inplace=True)

In [ ]:
df_three_ = df_three_[(df_three_['ddd_omsk'] != 'None') & (df_three_['ddd_omsk'] != "`CB")]

In [ ]:
df_three_.loc['2004-11-01':'2005-03-01'].tail(50)

In [ ]:
# time period

MONTH_START = 11
MONTH_END = 3
DAY_START = 1
DAY_END = 1

import datetime

df_three_winter = df_three_[datetime.date(1999,MONTH_START,1):datetime.date(2000,MONTH_END,DAY_END)]
df_three_winter = df_three_winter.append(df_three_[datetime.date(2000,MONTH_START,DAY_START):datetime.date(2001,MONTH_END,DAY_END)])
df_three_winter = df_three_winter.append(df_three_[datetime.date(2001,MONTH_START,DAY_START):datetime.date(2002,MONTH_END,DAY_END)])
df_three_winter = df_three_winter.append(df_three_[datetime.date(2002,MONTH_START,DAY_START):datetime.date(2003,MONTH_END,DAY_END)])
df_three_winter = df_three_winter.append(df_three_[datetime.date(2003,MONTH_START,DAY_START):datetime.date(2004,MONTH_END,DAY_END)])
df_three_winter = df_three_winter.append(df_three_[datetime.date(2004,MONTH_START,DAY_START):datetime.date(2005,MONTH_END,DAY_END)])
df_three_winter = df_three_winter.append(df_three_[datetime.date(2005,MONTH_START,DAY_START):datetime.date(2006,MONTH_END,DAY_END)])

In [ ]:
df_three_winter.groupby('ddd_omsk')['mean'].count()

In [ ]:
df_three_winter.groupby('ddd_omsk', as_index=False)[['diff', 'abs_diff']].mean()

In [ ]:
df_three_winter[df_three_winter['omsk'] <= -15].groupby('ddd_omsk', as_index=False)[['diff', 'abs_diff']].mean()

In [ ]:
df_three_winter[df_three_winter['omsk'] <= -20].groupby('ddd_omsk', as_index=False)[['diff', 'abs_diff']].mean()

In [ ]:
df_three_winter[df_three_winter['omsk'] <= -25].groupby('ddd_omsk', as_index=False)[['diff', 'abs_diff']].mean()

In [ ]:
df_three_winter[(df_three_winter['ddd_omsk'] == 'South')]['diff'].plot.kde(bw_method=0.1)

In [ ]:
df_three_winter[(df_three_winter['ddd_omsk'] == 'North')]['diff'].plot.kde(bw_method=0.1)

In [ ]:
# plot data
# import matplotlib.pyplot as plt
# fig, ax = plt.subplots(figsize=(15,7))
# use unstack()
# df_three_winter.groupby('ddd_omsk', as_index=False)[['diff', 'abs_diff']].mean().unstack('diff')

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df_three_winter[df_three_winter['ddd_omsk']=='East'].plot(y="diff",title='unit1',colormap='Reds_r')
df_three_winter[df_three_winter['ddd_omsk']=='North'].plot(y="diff",title='unit1',colormap='Reds_r')
df_three_winter[df_three_winter['ddd_omsk']=='South'].plot(y="diff",title='unit1',colormap='Reds_r')
df_three_winter[df_three_winter['ddd_omsk']=='West'].plot(y="diff",title='unit1',colormap='Reds_r')

In [ ]:
df_three_winter[df_three_winter['ddd_omsk'] == 'South']['diff'].plot.kde(bw_method=0.1)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(15,7))

ax = df_three_winter[df_three_winter['ddd_omsk'] == 'East']['diff'].plot(kind='kde',bw_method=0.2)
ax = df_three_winter[df_three_winter['ddd_omsk'] == 'North']['diff'].plot(kind='kde',bw_method=0.2)
ax = df_three_winter[df_three_winter['ddd_omsk'] == 'South']['diff'].plot(kind='kde',bw_method=0.2)
ax = df_three_winter[df_three_winter['ddd_omsk'] == 'West']['diff'].plot(kind='kde',bw_method=0.2)
ax.legend(["East", "North", "South", "West"]);

# df_three_winter[df_three_winter['ddd_omsk'] == 'South']['diff'].plot(kind='kde', secondary_y=True, ax=ax)
# df_three_winter[df_three_winter['ddd_omsk'] == 'North']['diff'].plot(kind='kde', secondary_y=True, ax=ax)
# df_three_winter[df_three_winter['ddd_omsk'] == 'East']['diff'].plot(kind='kde', secondary_y=True, ax=ax)
# df_three_winter[df_three_winter['ddd_omsk'] == 'West']['diff'].plot(kind='kde', secondary_y=True, ax=ax)


In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
ax = df_three_winter[(df_three_winter['ddd_omsk'] == 'East') & (df_three_winter['omsk'] <= -15)]['diff'].plot(kind='kde',bw_method=0.2)
ax = df_three_winter[(df_three_winter['ddd_omsk'] == 'North') & (df_three_winter['omsk'] <= -15)]['diff'].plot(kind='kde',bw_method=0.2)
ax = df_three_winter[(df_three_winter['ddd_omsk'] == 'South') & (df_three_winter['omsk'] <= -15)]['diff'].plot(kind='kde',bw_method=0.2)
ax = df_three_winter[(df_three_winter['ddd_omsk'] == 'West') & (df_three_winter['omsk'] <= -15)]['diff'].plot(kind='kde',bw_method=0.2)
ax.legend(["East", "North", "South", "West"]);

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
ax = df_three_winter[(df_three_winter['ddd_omsk'] == 'East') & (df_three_winter['omsk'] <= -20)]['diff'].plot(kind='kde',bw_method=0.2)
ax = df_three_winter[(df_three_winter['ddd_omsk'] == 'North') & (df_three_winter['omsk'] <= -20)]['diff'].plot(kind='kde',bw_method=0.2)
ax = df_three_winter[(df_three_winter['ddd_omsk'] == 'South') & (df_three_winter['omsk'] <= -20)]['diff'].plot(kind='kde',bw_method=0.2)
ax = df_three_winter[(df_three_winter['ddd_omsk'] == 'West') & (df_three_winter['omsk'] <= -20)]['diff'].plot(kind='kde',bw_method=0.2)
ax.legend(["East", "North", "South", "West"]);

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
ax = df_three_winter[(df_three_winter['ddd_omsk'] == 'East') & (df_three_winter['omsk'] <= -25)]['diff'].plot(kind='kde',bw_method=0.1)
ax = df_three_winter[(df_three_winter['ddd_omsk'] == 'North') & (df_three_winter['omsk'] <= -25)]['diff'].plot(kind='kde',bw_method=0.1)
ax = df_three_winter[(df_three_winter['ddd_omsk'] == 'South') & (df_three_winter['omsk'] <= -25)]['diff'].plot(kind='kde',bw_method=0.1)
ax = df_three_winter[(df_three_winter['ddd_omsk'] == 'West') & (df_three_winter['omsk'] <= -25)]['diff'].plot(kind='kde',bw_method=0.1)
ax.legend(["East", "North", "South", "West"]);

In [ ]:
df_three_winter[df_three_winter['omsk'] < -39]